In [1]:
import os
from dotenv import load_dotenv
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
load_dotenv()

AZURE_OPENAI_KEY = os.getenv("api_key")
AZURE_DEPLOYMENT_NAME = os.getenv("model-name")
API_VERSION = os.getenv("api-version")
AZURE_OPENAI_ENDPOINT = os.getenv("azure_endpoint")



# Create Azure Model Client
model_client = AzureOpenAIChatCompletionClient(
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_key=AZURE_OPENAI_KEY,
    api_version=API_VERSION,
    deployment_name=AZURE_DEPLOYMENT_NAME ,  # Your deployment name
    model =AZURE_DEPLOYMENT_NAME 
)

In [2]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.messages import AgentEvent, ChatMessage
from autogen_agentchat.teams import SelectorGroupChat
from autogen_agentchat.ui import Console


def weather_check_tool(city: str) -> str:
    weather_data = {
        "Dubai": "Sunny, 35°C",
        "New York": "Cloudy, 22°C",
        "London": "Rainy, 18°C",
        "Tokyo": "Clear, 26°C"
    }
    return weather_data.get(city, "Weather data not available.")

def currency_exchange_tool(amount: float, from_currency: str, to_currency: str) -> str:
    exchange_rates = {
        ("USD", "EUR"): 0.92,
        ("EUR", "USD"): 1.08,
        ("USD", "AED"): 3.67,
        ("AED", "USD"): 0.27
    }
    rate = exchange_rates.get((from_currency, to_currency), None)
    if rate:
        converted_amount = amount * rate
        return f"{amount} {from_currency} is equal to {converted_amount:.2f} {to_currency}."
    return "Exchange rate not available."



In [3]:

planning_agent = AssistantAgent(
    "PlanningAgent",
    description="An agent for planning tasks. It should break down tasks and delegate them to the appropriate agents.",
    model_client=model_client,
    system_message="""
    You are a planning agent.
    Your job is to break down complex tasks into smaller, manageable subtasks.
    Your team members are:
        WeatherAgent: Checks weather conditions
        CurrencyAgent: Handles currency conversion
    
    You only plan and delegate tasks - you do not execute them yourself.
    
    When assigning tasks, use this format:
    1. <agent> : <task>
    
    After all tasks are complete, summarize the findings and end with "TERMINATE".
    """,
)

weather_agent = AssistantAgent(
    "WeatherAgent",
    description="An agent that provides current weather conditions for a given city.",
    tools=[weather_check_tool],
    model_client=model_client,
    system_message="""
    You are a weather-checking agent.
    Your only tool is weather_check_tool - use it to fetch weather data for a city.
    """,
)

currency_agent = AssistantAgent(
    "CurrencyAgent",
    description="An agent that performs currency exchange calculations.",
    model_client=model_client,
    tools=[currency_exchange_tool],
    system_message="""
    You are a currency exchange agent.
    Your job is to convert a given amount from one currency to another using the available exchange rates.
    """,
)


In [4]:
# Mention all termination conditions

text_mention_termination = TextMentionTermination("TERMINATE")
max_messages_termination = MaxMessageTermination(max_messages=25)
termination = text_mention_termination | max_messages_termination

In [5]:
# Create the team

team = SelectorGroupChat(
    [planning_agent, weather_agent, currency_agent],
    model_client=model_client,
    termination_condition=termination,
    allow_repeated_speaker=True,  # Allow an agent to speak multiple turns in a row.
)

In [6]:
# Run the task
task = "What is the current weather in Dubai, and how much is 100 USD in AED?"
await Console(team.run_stream(task=task))

NotFoundError: Error code: 404 - {'error': {'code': '404', 'message': 'Resource not found'}}

In [6]:
# Custom selector function

from typing import Sequence

def selector_func(messages: Sequence[AgentEvent | ChatMessage]) -> str | None:
    if messages[-1].source != planning_agent.name:
        return planning_agent.name
    return None


In [7]:
await team.reset()
team = SelectorGroupChat(
    [planning_agent, weather_agent, currency_agent],
    model_client=model_client,
    termination_condition=termination,
    allow_repeated_speaker=True,
    selector_func=selector_func,
)

In [8]:
task = "What is the current weather in Dubai, and how much is 100 USD in AED?"
await Console(team.run_stream(task=task))

---------- TextMessage (user) ----------
What is the current weather in Dubai, and how much is 100 USD in AED?
---------- TextMessage (PlanningAgent) ----------
1. WeatherAgent: Check the current weather in Dubai.  
2. CurrencyAgent: Convert 100 USD to AED.  
---------- ToolCallRequestEvent (WeatherAgent) ----------
[FunctionCall(id='call_bOQUUIBjZB0LmQ65TdJoXu1m', arguments='{"city":"Dubai"}', name='weather_check_tool')]
---------- ToolCallExecutionEvent (WeatherAgent) ----------
[FunctionExecutionResult(content='Sunny, 35°C', name='weather_check_tool', call_id='call_bOQUUIBjZB0LmQ65TdJoXu1m', is_error=False)]
---------- ToolCallSummaryMessage (WeatherAgent) ----------
Sunny, 35°C
---------- TextMessage (PlanningAgent) ----------
1. CurrencyAgent: Convert 100 USD to AED.  
---------- ToolCallRequestEvent (CurrencyAgent) ----------
[FunctionCall(id='call_VeAASMMAaFy0Vpk8WDeTzg87', arguments='{"amount":100,"from_currency":"USD","to_currency":"AED"}', name='currency_exchange_tool')]
----

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='What is the current weather in Dubai, and how much is 100 USD in AED?', type='TextMessage'), TextMessage(source='PlanningAgent', models_usage=RequestUsage(prompt_tokens=130, completion_tokens=27), metadata={}, content='1. WeatherAgent: Check the current weather in Dubai.  \n2. CurrencyAgent: Convert 100 USD to AED.  ', type='TextMessage'), ToolCallRequestEvent(source='WeatherAgent', models_usage=RequestUsage(prompt_tokens=126, completion_tokens=16), metadata={}, content=[FunctionCall(id='call_bOQUUIBjZB0LmQ65TdJoXu1m', arguments='{"city":"Dubai"}', name='weather_check_tool')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='WeatherAgent', models_usage=None, metadata={}, content=[FunctionExecutionResult(content='Sunny, 35°C', name='weather_check_tool', call_id='call_bOQUUIBjZB0LmQ65TdJoXu1m', is_error=False)], type='ToolCallExecutionEvent'), ToolCallSummaryMessage(source='WeatherAgent',